<a href="https://colab.research.google.com/github/jlopetegui98/RebornToBeWilde/blob/main/5-Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Experiments with Mistral7B-Instruct (baseline) and fine tuned model with Oscard Wilde texts**

The idea of the experiments consists in using the Classifier trained for authorship attribution (BertAA_Wilde_vs_Mistral) to test the capacity of the fine tuned model to mimetize Wilde's writting style.

Then, for each model we are going to generate fiction stories from the same starting lines and assign labels to each story and compare the results between the two models. The hypothesis is that for the fine tuned model the capacity of the model to distinguish between Wilde and Mistral will decay.

In [1]:
# uncomment the following lines to run in google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install the required packages if working in google colab
!pip install -U simpletransformers
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 100

In [4]:
# import the required packages
import torch
import simpletransformers
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, platform, gradio, warnings
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from huggingface_hub import notebook_login
import json
from tqdm import tqdm
import utils

In [5]:
from utils import *

In [6]:
# data and models paths
# dir_root = '.' # comment this line to run in google colab
dir_root = './drive/MyDrive/RebornToBeWilde' # uncomment this line to run in google colab
dir_data = f'{dir_root}/data'
models_path = f'{dir_root}/models'
clf_path = f'./drive/MyDrive/DL-ENS/models/BertClassifier(BERTAA)_wilde_vs_mistral7B.pt'
list_to_generate_path = f'{dir_data}/story_prompts_for_evaluation.txt'
ft_model = f'{dir_root}/models/Mistral7B_fine_tuned_OscarWilde.pt'

In [7]:
# load classifier (wilde vs mistral7B-Instruct)
clf = torch.load(clf_path)
clf

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [8]:
#base model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [9]:
# load the lines to start the generation
texts_to_generation = []
with open(list_to_generate_path, 'r+', encoding='utf-8') as fd:
  texts_to_generation = fd.readlines()
texts_to_generation = [text[:-1] for text in texts_to_generation]

**Experiments with baseline model**

In [9]:
model = load_model(model_name)
tokenizer = load_tokenizer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [10]:
from utils import *

In [11]:
from tqdm import tqdm

In [14]:
import regex as re

In [13]:
def clf_exp_(model, tokenizer, clf, texts):
    """
    Function for the experiments: Given a model, a tokenizer and a classifier, it generates text using the
    lines in the texts list and then predicts the label of the generated text using the classifier
    inputs:
      model: model: model to use
      tokenizer: tokenizer: tokenizer to use
      clf: classifier: classifier to use
      texts: list: list of strings with the lines to generate and predict
    outputs:
        label_predictions: list: list of predictions of the classifier
        generated_texts: list: list of the generated texts
    """

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # pattern to remove the prompt tokens after generation
    patt = r'\[INST]|\[\/INST]|\<s>|\</s>|This are the first lines of a work of fiction. Continue it.'

    generated_texts = [] # list of generated texts
    label_predictions = [] # list of predictions of the classifier
    for input in tqdm(texts):
      tokens = tokenize(tokenizer, input)
      model_inputs = tokens.to(device)
      generated_ids = model.generate(**model_inputs, max_new_tokens=500, do_sample=True)
      decoded = tokenizer.batch_decode(generated_ids)
      decoded = [re.sub(patt, '', x) for x in decoded] # clean the generated texts
      preds, _ = clf.predict(decoded) # predict the label of the generated text with clf
      label_predictions.extend(preds)
      generated_texts.extend(decoded)
      del model_inputs
      del decoded
      del generated_ids
    return label_predictions, generated_texts

In [15]:
author_preds, generated_texts = clf_exp_(model, tokenizer, clf, texts_to_generation[:20])

  0%|          | 0/20 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:36<11:41, 36.93s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0it [00:00, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

 95%|█████████▌| 19/20 [06:38<00:15, 15.51s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 20/20 [06:56<00:00, 20.82s/it]


In [16]:
print(f"Predictions:\n Wilde: {author_preds.count(0)/len(author_preds) * 100} % \n Mistral7B: {author_preds.count(1)/len(author_preds) * 100} %")

Predictions:
 Wilde: 0.0 % 
 Mistral7B: 100.0 %


In [17]:
def save_generated_texts_and_labels_(texts, labels, model = 'baseline', data_path = 'data'):
    """
    Save the generated texts and labels in a json file
    inputs:
      texts: list: list of generated texts
      labels: list: list of labels of the generated texts
      model: str: name of the model used to generate the texts
    """
    dict_text_to_author = {'text': [], 'label': []}

    for i in range(len(texts)):
      dict_text_to_author['text'].append(texts[i])
      dict_text_to_author['label'].append(str(labels[i]))

    with open(f"{data_path}/{model}_generated_texts.json", 'w+') as fd:
      json.dump(dict_text_to_author, fd)

In [24]:
# saver the results
import json
save_generated_texts_and_labels_(generated_texts, author_preds, model = 'baseline', data_path = dir_data)

**Experiments with fine-tuned model**

In [10]:
model = load_model(model_name, adapt = True, from_finetuned = True, model_path = ft_model)
tokenizer = load_tokenizer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [11]:
import re

In [14]:
author_preds_ft, generated_texts_ft = clf_exp_(model, tokenizer, clf, texts_to_generation)

  0%|          | 0/100 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 1/100 [00:50<1:22:36, 50.06s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0it [00:00, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

 99%|█████████▉| 99/100 [1:17:26<00:47, 47.57s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 100/100 [1:18:14<00:00, 46.94s/it]


In [15]:
print(f"Predictions:\n Wilde: {author_preds_ft.count(0)/len(author_preds_ft) * 100} % \n Mistral7B: {author_preds_ft.count(1)/len(author_preds_ft) * 100} %")

Predictions:
 Wilde: 55.00000000000001 % 
 Mistral7B: 45.0 %


In [18]:
# saver the results
save_generated_texts_and_labels_(generated_texts_ft, author_preds_ft, model = 'Mistral7B_fine_tuned', data_path = dir_data)

In [ ]:
print("Done")